In [1]:
import findspark
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf #numpy needs to be 1.23 or earlier in order for tf to import correctly
from pyspark.sql import Row
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
findspark.init()

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataCleaning").getOrCreate()

In [3]:
from pyspark import SparkFiles
file = "income.csv"
spark.sparkContext.addFile(file)
df = spark.read.csv(SparkFiles.get("income.csv"), sep=",", header=True)
df.createOrReplaceTempView('income_spark')

In [4]:
df.show()

+---+----------------+------+------------+-------------+------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|   education|education_num|    marital_status|       occupation|  relationship| race|   sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+---+----------------+------+------------+-------------+------------------+-----------------+--------------+-----+------+------------+------------+--------------+--------------+------+
| 40|Self-emp-not-inc|223881| Prof-school|           15|Married-civ-spouse|   Prof-specialty|       Husband|White|  Male|       99999|           0|            70| United-States|  >50K|
| 30|         Private|149118|     HS-grad|            9|          Divorced|     Craft-repair| Not-in-family|White|Female|           0|           0|            40| United-States| <=50K|
| 46|         Private|109209|Some-college|           10|Married-civ-spouse|

In [5]:

#using spark.sql pull all the columsn from the df except for capital.gains, capital.loss, and fnlwgt
df = spark.sql("""SELECT age, 
               workclass, 
               education, 
               education_num, 
               marital_status, 
               occupation, 
               relationship, 
               race, 
               sex, 
               hours_per_week, 
               native_country, 
               income
               FROM income_spark""")




In [6]:
#make the spark dataframe df a pandas dataframe
income = df.toPandas()

In [7]:
income.replace("?", np.nan, inplace=True)
income.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
0,40,Self-emp-not-inc,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,70,United-States,>50K
1,30,Private,HS-grad,9,Divorced,Craft-repair,Not-in-family,White,Female,40,United-States,<=50K
2,46,Private,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,40,United-States,>50K
3,32,Private,Assoc-voc,11,Married-civ-spouse,Other-service,Husband,White,Male,60,United-States,>50K
4,54,NaN,Preschool,1,Married-civ-spouse,NaN,Wife,White,Female,40,Mexico,<=50K


In [8]:
income.dropna(inplace=True)
income.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,hours_per_week,native_country,income
0,40,Self-emp-not-inc,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,70,United-States,>50K
1,30,Private,HS-grad,9,Divorced,Craft-repair,Not-in-family,White,Female,40,United-States,<=50K
2,46,Private,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,40,United-States,>50K
3,32,Private,Assoc-voc,11,Married-civ-spouse,Other-service,Husband,White,Male,60,United-States,>50K
5,63,Private,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,16,United-States,<=50K


In [9]:
income.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23149 entries, 0 to 24999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             23149 non-null  object
 1   workclass       23149 non-null  object
 2   education       23149 non-null  object
 3   education_num   23149 non-null  object
 4   marital_status  23149 non-null  object
 5   occupation      23149 non-null  object
 6   relationship    23149 non-null  object
 7   race            23149 non-null  object
 8   sex             23149 non-null  object
 9   hours_per_week  23149 non-null  object
 10  native_country  23149 non-null  object
 11  income          23149 non-null  object
dtypes: object(12)
memory usage: 2.3+ MB


In [10]:
income["age"]=pd.to_numeric(income["age"])
income["education_num"]=pd.to_numeric(income["education_num"])
income["hours_per_week"]=pd.to_numeric(income["hours_per_week"])

In [11]:
income.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23149 entries, 0 to 24999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             23149 non-null  int64 
 1   workclass       23149 non-null  object
 2   education       23149 non-null  object
 3   education_num   23149 non-null  int64 
 4   marital_status  23149 non-null  object
 5   occupation      23149 non-null  object
 6   relationship    23149 non-null  object
 7   race            23149 non-null  object
 8   sex             23149 non-null  object
 9   hours_per_week  23149 non-null  int64 
 10  native_country  23149 non-null  object
 11  income          23149 non-null  object
dtypes: int64(3), object(9)
memory usage: 2.3+ MB


In [12]:
income_cat = income.dtypes[income.dtypes == 'object'].index.tolist()

In [13]:
#check the number of unique values in each column
income[income_cat].nunique()

workclass          7
education         16
marital_status     7
occupation        14
relationship       6
race               5
sex                2
native_country    40
income             2
dtype: int64

In [14]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(income[income_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(income_cat)
encode_df.head()

,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,...,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income_<=50K,income_>50K
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [15]:
# Merge one-hot encoded features and drop the originals
income = income.merge(encode_df,left_index=True, right_index=True)
income = income.drop(income_cat,axis=1)
income.head()

,age,education_num,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,...,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income_<=50K,income_>50K
0,40,15,70,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,30,9,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,46,10,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,32,11,60,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,63,10,16,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
#drop income_>50K because there are 2 income columns
income.drop(['income_>50K'], axis=1, inplace=True)


In [17]:
# Split our preprocessed data into our features and target arrays
y = income["income_<=50K"].values #target is a binary 1 or 0 response
X = income.drop(["income_<=50K"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 20


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 16191 (63.25 KB)
Trainable params: 16191 (63.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
503/503 [==============================] - 1s 734us/step - loss: 0.4005 - accuracy: 0.8113
Epoch 2/100
503/503 [==============================] - 0s 735us/step - loss: 0.3652 - accuracy: 0.8319
Epoch 3/100
503/503 [==============================] - 0s 706us/step - loss: 0.3564 - accuracy: 0.8337
Epoch 4/100
503/503 [==============================] - 0s 705us/step - loss: 0.3530 - accuracy: 0.8350
Epoch 5/100
503/503 [==============================] - 0s 701us/step - loss: 0.3483 - accuracy: 0.8379
Epoch 6/100
503/503 [==============================] - 0s 710us/step - loss: 0.3438 - accuracy: 0.8389
Epoch 7/100
503/503 [==============================] - 0s 700us/step - loss: 0.3401 - accuracy: 0.8407
Epoch 8/100
503/503 [==============================] - 0s 713us/step - loss: 0.3357 - accuracy: 0.8418
Epoch 9/100
503/503 [==============================] - 0s 704us/step - loss: 0.3326 - accuracy: 0.8443
Epoch 10/100
503/503 [==============================] - 0s 728us/step - l

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)


168/168 - 0s - loss: 1.1973 - accuracy: 0.7782 - 191ms/epoch - 1ms/step
Loss: 1.1972652673721313, Accuracy: 0.7781926989555359


In [24]:

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 1.1972652673721313, Accuracy: 0.7781926989555359
